<a href="https://colab.research.google.com/github/tirabo/Matematica-Discreta-I/blob/master/Test_de_primalidad_de_Miller_Rabin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Método binario para exponenciacion modular

Ver: [https://es.wikipedia.org/wiki/Exponenciaci%C3%B3n_modular](https://es.wikipedia.org/wiki/Exponenciaci%C3%B3n_modular)

Sean `a, d, n` enteros positivos se desea calcular `a**d % n`
1.  Se calcula la expresión binaria de $d$. 
Si $d$ en base $2$ es $d_m...d_0$, entonces
$$d = \sum_{i=0}^{m} d_i \cdot  2^i$$
Entonces
\begin{align*}
a^d &= \prod_{i=0}^{m} a^{d_i \cdot 2^i} \\ 
             &= \prod_{i=0}^{m} (a^{2^i})^{d_i}.
\end{align*} 
Usando lo anterior podemos calcular `c = a**d % n` recursivamente:
        c = (a**(2**0))**d_0 % n        (caso base)
        c = c * (a**(2**i))**d_i % n    (paso i)
  
3. El cálculo anterior no produce gran beneficio en la eficiencia de la exponenciación modular, pues `a**(2**i) % n` sigue siendo costoso de calcular. Utilizando la idea de escribir el exponente en base 2 se puede obtener un algoritmo eficiente. La idea para hacer un algoritmo eficiente es la siguiente:
$$   d = d_0 \cdot 2 + r  \quad  \Rightarrow \quad  a^d \equiv (a^2)^{d_0}  a^r \pmod n \tag{1}.$$
Si $a^2 \equiv a_0 \pmod n$, obtenemos
$$   d = d_0 \cdot 2 + r  \quad  \Rightarrow \quad  a^d \equiv {a_0}^{d_0}  a^r \pmod n \tag{2}.$$ 
Supongamos que queremos calcular $r$ tal que 
$$3^{9} \equiv r\; (\operatorname{mod} 5),$$
con $0 \le r < 5$. Esto no es más que calcular
         3**9 % 5.
En  este caso, podemos hacer este cálculo en Python directamente, pero mostraremos como es el método para ejemplificar. Lo  que haremos es lo siguiente: como `9 = 2*4 + 1`, tenemos por la fórmula (2) que 
        3**9 % 5 = (3**2)**4 * 3 % 5
                 =      4**4 * 3 % 5     # 9 % 5 = 4
                 = (4**2)**2 * 3 % 5
                 =      1**2 * 3 % 5     # 16 % 5 = 1
                 = 3 % 5
Supongamos ahora que queremos calcular 
         5**1125899986842625 % 100000037.
Hacer este cálculo en Python directamente no nos da un resultado satisfactorio. Haga el intento para  convencerse. Pero  como `1125899986842625 < 2**51`, usando el método de las fórmulas (1) y (2) necesitaremos alrededor de 50 pasos del tipo 
        c = x**2 * 5**r * c % 100000037
donde `0 <= x, c <= 100000037` y `r = 0, 1`. La clave es que cada uno de estos cálculos se realiza con facilidad en Python. 

**Ejercicio 1.** Sean `a, d, n` enteros positivos definir la función

        pot_mod(a, d, n) 

que devuelve `a**d % n` (`a` a la `d` módulo `n`) usando el  método binario de exponenciación modular.

In [ ]:
# Escribir más abajo (la linea punteada debe ser reemplazada por lineas de código)

def pot_mod(a: int, d: int, n: int) -> int:
    # pre: a, d, n enteros positivos
    # post: devuelve a**d % n calculado por el método binario de exponenciacion modular
    ...


In [ ]:
# Tests (los resultados deberían ser casi instantaneos)
print(pot_mod(2,4,15)) # 1
print(pot_mod(7, 385, 11)) # 10
print(pot_mod(5,1125899986842625, 100000037 )) # 98770120


## Test de primalidad de  Miller-Rabin 

El test de primalidad Miller-Rabin es una prueba probabilística de primalidad: un algoritmo que determina si un número dado es probable que sea primo.

Este test sigue siendo ampliamente utilizado en la práctica (en RSA, por ejemplo) y es una de las pruebas más simples y rápidas conocidas.

Dado $m$ un entero positivo.
- Si le hacemos el test a $m$ y no supera la prueba, entonces el número no es primo. 
- Si hacemos $k$ veces el test y $m$ supera las $k$ pruebas, entonces  $m$ tiene la probabilidad $1 - ( 1 / 4^k)$ de ser primo. 

Ver: [https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test)

**Números probablemente primos**

Sea $n > 2$ un entero impar, entonces $n = 2^s \cdot d + 1$ con $d$ impar. Sea $a$ entero  tal que $0 < a < n$. En tonces diremos que $n$ es _fuertemente probablemente primo (FPP) respecto a la base_ $a$ si se cumple  
- $a^{d} \equiv 1 \pmod{n}$, o
- $a^{2^r\cdot\, d} \equiv -1 \pmod{n}$  para algún $r$ tal que $0 \le r < s$.

Con la aplicación del teorema de Fermat y la ecuación lineal de congruencia no es difíl probar que todo número primo es FPP respecto a cualquier base. El contrarrecíproco de esta afimación nos dice que un número que no es FPP respecto a alguna base es compuesto.

Ahora bien, un número $n$ que es FPP respecto a todas las bases $ 0 < a < n$ es primo, pero este cáclulo es computacionalmente imposible para primos grandes.  

Por otro lado, un número $n$ que es FPP respecto alguna base $ 0 < a < n$ podría ser compuesto, pero hay una probabilidad mayor que 0.75 de que sea primo. La verificación con diferentes bases de que un número es FPP acerca a 1 la probabilidad de que el número sea primo.  


**Algoritmo**

El test  de primalidad de Miller-Rabin se basa en las observaciones realizadas más arriba: sea `n` entero positivo impar entonces, `n = 2**s * d + 1` con `d` impar, y sea `k` entero positivo.  
1. Elegir al  azar `a` entero tal que `0 < a < n`.
2. Verificar que  `n` es FPP respecto a la base `a`.
3. Repetir 1. y 2. `k` veces.

Si `n` es FPP las `k` veces,  entonces decimos que `n` supera el test de primalidad de Miller-Rabin (y lo consideramos primo).  

_Observación ._ El test de primalidad de Miller-Rabín requiere el orden de $k \cdot m^3$ operaciones, donde $m$ es el tamaño o cantidad de dígitos del número. 

**Ejercicio 2**

1. Escribir la función que determina  si un número `n` es fuertemente probablemente primo respecto a una base `a`.
2. Escribir el test de primalidad de Miller-Rabin.    

In [ ]:
import random

# Escribir más abajo (las lineas punteadas deben ser reemplazadas por lineas de código)

def pot2(n: int):
  # pre: n > 0, n impar
  # post: devuelve (s, d) tal n = 2**s * d + 1,  con d impar.
  ...
  return (s, d)

def fpp(n, a: int) -> bool:
  # pre:  n impar, 0 < a < n
  # post: devuelve True si n = es FPP respecto a a. False en caso contrario
  (s, d) = pot2(n)
  ...

def test_Miller_Rabin(n: int, k: int) -> bool:
  # pre: n > 2, n impar, k > 0
  # post: si n es FPP k-veces (con base al azar) devuelve True. En  caso  contrario  devuelve False. 
  ...

_Observación._ Para la definición de `test_Miller_Rabin()` puede usarse  la función `fpp()` `k`-veces aunque esto no sería muy conveniente (¿por qué?). Si lo desea, puede reformular la modularización con funciones auxiliares (lo más elegante) o simplemente reescribir parte del código de `fpp()` dentro de `test_Miller_Rabin()`.

In [ ]:
# Tests: si las funciones están bien implementadas lo siguiente debería ser muy rápido. 

print(fpp(17, 5)) # True
print(fpp(3221225473, 53)) # True

primo = True
for i in range(27):
  primo = primo and fpp(27, i)
print(primo) # False

print(test_Miller_Rabin(31, 4)) # True
print(test_Miller_Rabin(351, 10)) # probablemente False
print(test_Miller_Rabin(10**8+37, 5)) # True
print(test_Miller_Rabin(2074722246773485207821695222107608587480996474721117292752992589912196684750549658310084416732550077, 100)) # True
print(test_Miller_Rabin(323000000000023902000000000442187, 100)) # False (probablemente)


## Comentario final

Los primos _grandes_ son muy importantes para el uso en criptografía de clave pública, en particular para el algoritmo RSA. 

El objetivo es calcular de manera eficiente números primos aleatorios muy grandes con un tamaño de bit específico. 

El método estándar para implementar un generador de números primos aleatorios se da a continuación:

1. Preseleccione un número aleatorio con el tamaño de bits deseado 
2.Asegúrese de que el número elegido no sea divisible por los primeros cientos de números primos (estos están pregenerados)
3. Aplique un cierto número de iteraciones de la prueba de primalidad de Rabin-Miller. Un número más que aceptable son 100 repeticiones. 